# Notes on Phase Diversity

## Instrument model

The monochromatic optical point spread function (PSF) at wavelengh $\lambda$, denoted $h_\lambda$, is computed from the inverse Fourier transform of the complex pupil function (CPF).
The CPF is obtained from the binary pupil mask, $P$, and the wave front error (WFE), $\Phi$, which is itself modeled as a series of Zernike polynomials.

$$\begin{align}
h_\lambda(x, y) &= |\mathcal{F}^{-1}[P \, \exp(-j2\pi/\lambda\cdot\Phi)](x, y)|^2 \\
\Phi(u, v) &= \sum_k \alpha_k Z_k(u, v)
\end{align}$$

where $(x, y)$ denote image plane coordinates, while $(u, v)$ denote puil plane coordinates.
The $\alpha_k$'s are the Zernike coefficients and $Z$ is the Zernike basis.
Note that $\Phi$ is wavelength-independent.

The monochromatic system PSF $g_\lambda$ is obtained by convolving the monochromatic optical PSF with other (achromatic) PSFs, namely the detector PSF, $h_\text{det}$ and the attitude and orbit PSF (which arises from the motion of the satellite), $h_\text{ao}$:

$$\begin{align}
g_\lambda(x, y) &= (h_\lambda * h_\text{det} * h_\text{ao})(x, y)
\end{align}$$

The so-called broadband PSF $g_S$ associated to a point source of spectrum $S(\lambda)$ is:

$$\begin{align}
g_S(x, y) &= \int_\lambda g_\lambda(x, y) S(\lambda) \, \text{d} \lambda
\end{align}$$

## Numerical computation

The pupil mask $P$ is a square image essentially made of zeros and ones, except at edges, where it is smoothed for anti-aliasing purpose.
For Euclid, the mask is of side 1024, and the pupil of side 512.

The Zernike basis is precomputed up to the 8-th order, thus resulting in a $512 \times 512$ map of vectors of $45$ values,
stored as a $45 \times 1024 \times 1024$ cube in order to ensure data locality in memory throughout expensive computations.

The complex exponential is effectively computed only where $P$ is non null.

At wavelength $\lambda \, \text{nm}$, Each PSF pixel spans over $\lambda / 1024 \, \text{nm}$.
Euclid wavelengths span from 500 to 900 nm, and therefore a pixel measures roughly 0.5 to 0.9 nm.